In [1]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
FILENAME = '/Users/shanewang/Desktop/anran/Mar.xlsx'
SHEETNAME = '错号率-到项目'
SHEETNAME = SHEETNAME if SHEETNAME in pd.ExcelFile(FILENAME).sheet_names else 1
print("Using sheet: ", SHEETNAME)
df = pd.read_excel(FILENAME, sheet_name=SHEETNAME) #一级考核-错号率
raw = df.copy(deep=True)
raw.head()

Using sheet:  错号率-到项目


,行标签,S2题甄别失败,S4a题甄别失败,S4b题甄别失败,S6a题甄别失败,S6题甄别失败,S7题甄别失败,成功,关机/占线/无法接通/语音信箱,过滤问卷拒访,开场白拒访,空号/号码过期,配额已满,审核作废,时间不宜,停机,未购买此楼盘,未接触,无此人/打错了,无人接听,语言不通,直接挂断/被访者不说话,重复访问,主体问卷拒访,准业主-提前交付（已交付）,总计,接触量,错号量,接通量,错号率
0,集团,26.0,28.0,58.0,4.0,19.0,28.0,4965.0,2254.0,43.0,1299.0,238.0,7.0,2.0,651.0,73.0,41.0,8235.0,129.0,4591.0,38.0,413.0,58.0,386.0,9.0,23595,15360,507,8515,0.033008
1,上海区域,8.0,2.0,9.0,2.0,1.0,8.0,614.0,354.0,4.0,190.0,31.0,2.0,1.0,112.0,12.0,3.0,896.0,28.0,723.0,3.0,94.0,15.0,50.0,NaN,3162,2266,82,1189,0.036187
2,南京,NaN,NaN,NaN,NaN,NaN,NaN,33.0,12.0,NaN,5.0,1.0,1.0,NaN,6.0,NaN,NaN,9.0,NaN,20.0,NaN,2.0,NaN,1.0,NaN,90,81,1,49,0.012346
3,滁州凯迪融创玉兰公馆,NaN,NaN,NaN,NaN,NaN,NaN,24.0,12.0,NaN,2.0,1.0,1.0,NaN,6.0,NaN,NaN,9.0,NaN,18.0,NaN,2.0,NaN,1.0,NaN,76,67,1,37,0.014925
4,准业主,NaN,NaN,NaN,NaN,NaN,NaN,24.0,12.0,NaN,2.0,1.0,1.0,NaN,6.0,NaN,NaN,9.0,NaN,18.0,NaN,2.0,NaN,1.0,NaN,76,67,1,37,0.014925


In [3]:
def simplify(raw: pd.DataFrame):    
    labels = ['行标签', '总计', '接触量', '接触数','错号数', '错号量']
    for name in raw.columns.tolist():
        if name == '错号数':
            raw.rename(columns={'错号数':'错号量'}, inplace=True)
        if name == '接触数':
            raw.rename(columns={'接触数':'接触量'}, inplace=True)
        if name not in labels:
            raw.drop(name, axis=1, inplace=True)

        
simplify(raw)

raw.head(10)

,行标签,总计,接触量,错号量
0,集团,23595,15360,507
1,上海区域,3162,2266,82
2,南京,90,81,1
3,滁州凯迪融创玉兰公馆,76,67,1
4,准业主,76,67,1
5,1-准业主1,6,6,0
6,2-准业主2,70,61,1
7,滁州凯迪融创御园,13,13,0
8,准业主,13,13,0
9,2-准业主2,13,13,0


In [4]:
# raw.to_excel('sampleTest.xlsx', index=False)

In [5]:
city_names = []
deprecated_names = ['集团', '上海区域']
# deprecated_names = ['集团']
for name in raw['行标签'].tolist():
    if name not in deprecated_names and '准业主' not in name \
                                    and '磨合期' not in name \
                                    and '老业主' not in name \
                                    and '稳定期' not in name:
        city_names.append(name)

print(city_names)
len(city_names)

['南京', '滁州凯迪融创玉兰公馆', '滁州凯迪融创御园', '南京融创中南御园', '南通', '南通江语海', '南通九里桃源', '南通玖樟台', '南通玉兰公馆', '启东海上桃源', '启东桃源里', '内蒙', '呼和浩特瀚海壹號', '浦东', '上海桃花源', '商置', '上海香溢花城', '上海', '上海滨江壹号院', '上海葛洲坝虹桥玫瑰公馆', '上海领馆壹号院', '上海山水拾间', '上海壹号公馆', '苏北公司', '淮安融创广场', '淮南玉兰公馆', '徐州枫丹公馆', '徐州凤鸣桃源', '徐州凤鸣桃源风雅颂', '徐州淮海壹号', '徐州玉兰公馆', '枣庄御园', '苏南公司', '常州滨湖御园', '常州御园一二三期', '江阴江南御园', '靖江玉兰公馆', '苏州泰兴御园', '泰兴桃源府南苑', '泰州融创玉兰花园', '无锡瑷颐湾', '无锡大塘御园', '无锡东方御园', '无锡枫丹御园', '无锡惠山国际社区', '无锡惠山映', '无锡江阴玉兰公馆', '无锡玖里映月', '无锡满堂院', '无锡森邻森邻', '无锡文旅城', '无锡西羲里', '无锡香樟园', '无锡壹号院', '无锡玉兰公馆', '无锡运河壹号府', '无锡长江映', '苏中公司', '芜湖玉兰公馆', '镇江观澜公馆', '镇江御园', '苏州', '昆山玉兰公馆', '苏州泊云庭', '苏州浦西玫瑰园', '苏州太湖御园', '苏州壹号院', '张家港滨江御园', '新疆', '乌鲁木齐江南桃源']


70

In [6]:
row_records = []
for _, row in raw.iterrows():
    try:
        row_records.append([row['行标签'], row['接触量'], row['错号量']])
    except Exception as e:
        print(e)
#         row_records.append([row['行标签'], row['总计'], row['错号量']])
    # print(row['行标签'], row['总计'], row['错号数'])
row_records

[['集团', 15360, 507],
 ['上海区域', 2266, 82],
 ['南京', 81, 1],
 ['滁州凯迪融创玉兰公馆', 67, 1],
 ['准业主', 67, 1],
 ['1-准业主1', 6, 0],
 ['2-准业主2', 61, 1],
 ['滁州凯迪融创御园', 13, 0],
 ['准业主', 13, 0],
 ['2-准业主2', 13, 0],
 ['南京融创中南御园', 1, 0],
 ['准业主', 1, 0],
 ['1-准业主1', 1, 0],
 ['南通', 160, 1],
 ['南通江语海', 4, 0],
 ['准业主', 4, 0],
 ['1-准业主1', 4, 0],
 ['南通九里桃源', 126, 0],
 ['准业主', 126, 0],
 ['1-准业主1', 1, 0],
 ['2-准业主2', 125, 0],
 ['南通玖樟台', 17, 0],
 ['准业主', 17, 0],
 ['1-准业主1', 17, 0],
 ['南通玉兰公馆', 1, 0],
 ['准业主', 1, 0],
 ['1-准业主1', 1, 0],
 ['启东海上桃源', 4, 0],
 ['准业主', 4, 0],
 ['2-准业主2', 4, 0],
 ['启东桃源里', 8, 1],
 ['准业主', 8, 1],
 ['1-准业主1', 4, 1],
 ['2-准业主2', 4, 0],
 ['内蒙', 18, 0],
 ['呼和浩特瀚海壹號', 18, 0],
 ['准业主', 18, 0],
 ['1-准业主1', 18, 0],
 ['浦东', 4, 0],
 ['上海桃花源', 4, 0],
 ['磨合期', 3, 0],
 ['3-磨合期1', 3, 0],
 ['准业主', 1, 0],
 ['1-准业主1', 1, 0],
 ['商置', 1, 0],
 ['上海香溢花城', 1, 0],
 ['磨合期', 1, 0],
 ['4-磨合期2', 1, 0],
 ['上海', 40, 1],
 ['上海滨江壹号院', 1, 1],
 ['磨合期', 1, 1],
 ['4-磨合期2', 1, 1],
 ['上海葛洲坝虹桥玫瑰公馆', 19, 0],
 ['准业主', 19, 0],
 [

In [7]:
wrong2city = {}
city_appeared = []
cities = []
slice_idx = []
zyz = []
mhq = []
wdq = []
lyz = []

for i in range(0, len(row_records)):
    tag = row_records[i][0]
    if tag in city_names:
        wrong2city[row_records[i][2]] = tag
        city_appeared.append(tag)
        slice_idx.append(i)
slice_idx.append(len(row_records))
for i in range(len(slice_idx)-1):
    cities.append(row_records[slice_idx[i]:slice_idx[i+1]])
print(cities[0])    
print(city_appeared)
print(slice_idx)

[['南京', 81, 1]]
['南京', '滁州凯迪融创玉兰公馆', '滁州凯迪融创御园', '南京融创中南御园', '南通', '南通江语海', '南通九里桃源', '南通玖樟台', '南通玉兰公馆', '启东海上桃源', '启东桃源里', '内蒙', '呼和浩特瀚海壹號', '浦东', '上海桃花源', '商置', '上海香溢花城', '上海', '上海滨江壹号院', '上海葛洲坝虹桥玫瑰公馆', '上海领馆壹号院', '上海山水拾间', '上海壹号公馆', '苏北公司', '淮安融创广场', '淮南玉兰公馆', '徐州枫丹公馆', '徐州凤鸣桃源', '徐州凤鸣桃源风雅颂', '徐州淮海壹号', '徐州玉兰公馆', '枣庄御园', '苏南公司', '常州滨湖御园', '常州御园一二三期', '江阴江南御园', '靖江玉兰公馆', '苏州泰兴御园', '泰兴桃源府南苑', '泰州融创玉兰花园', '无锡瑷颐湾', '无锡大塘御园', '无锡东方御园', '无锡枫丹御园', '无锡惠山国际社区', '无锡惠山映', '无锡江阴玉兰公馆', '无锡玖里映月', '无锡满堂院', '无锡森邻森邻', '无锡文旅城', '无锡西羲里', '无锡香樟园', '无锡壹号院', '无锡玉兰公馆', '无锡运河壹号府', '无锡长江映', '苏中公司', '芜湖玉兰公馆', '镇江观澜公馆', '镇江御园', '苏州', '昆山玉兰公馆', '苏州泊云庭', '苏州浦西玫瑰园', '苏州太湖御园', '苏州壹号院', '张家港滨江御园', '新疆', '乌鲁木齐江南桃源']
[2, 3, 7, 10, 13, 14, 17, 21, 24, 27, 30, 34, 35, 38, 39, 44, 45, 48, 49, 52, 55, 58, 61, 64, 65, 68, 71, 75, 78, 81, 84, 87, 91, 92, 96, 99, 102, 106, 110, 113, 117, 120, 124, 127, 130, 134, 137, 140, 143, 146, 149, 153, 156, 159, 162, 165, 171, 174, 175, 178, 182, 186, 187, 192, 195, 199, 202, 208, 211

In [8]:
for city_info in cities:
    CITY = city_info[0][0]
    for line in city_info[1:]:
        if '准业主' in line[0] and '准业主' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            zyz.append(line)
        if '磨合期' in line[0] and '磨合期' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            mhq.append(line)
        if '稳定期' in line[0] and '稳定期' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            wdq.append(line)
        if '老业主' in line[0] and '老业主' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            lyz.append(line)
zyz

[['滁州凯迪融创玉兰公馆-1-准业主1', 6, 0],
 ['滁州凯迪融创玉兰公馆-2-准业主2', 61, 1],
 ['滁州凯迪融创御园-2-准业主2', 13, 0],
 ['南京融创中南御园-1-准业主1', 1, 0],
 ['南通江语海-1-准业主1', 4, 0],
 ['南通九里桃源-1-准业主1', 1, 0],
 ['南通九里桃源-2-准业主2', 125, 0],
 ['南通玖樟台-1-准业主1', 17, 0],
 ['南通玉兰公馆-1-准业主1', 1, 0],
 ['启东海上桃源-2-准业主2', 4, 0],
 ['启东桃源里-1-准业主1', 4, 1],
 ['启东桃源里-2-准业主2', 4, 0],
 ['呼和浩特瀚海壹號-1-准业主1', 18, 0],
 ['上海桃花源-1-准业主1', 1, 0],
 ['上海葛洲坝虹桥玫瑰公馆-2-准业主2', 19, 0],
 ['上海领馆壹号院-1-准业主1', 1, 0],
 ['上海山水拾间-2-准业主2', 14, 0],
 ['上海壹号公馆-2-准业主2', 5, 0],
 ['淮安融创广场-1-准业主1', 4, 0],
 ['淮南玉兰公馆-1-准业主1', 16, 0],
 ['徐州枫丹公馆-1-准业主1', 34, 2],
 ['徐州枫丹公馆-2-准业主2', 20, 1],
 ['徐州凤鸣桃源-1-准业主1', 1, 0],
 ['徐州凤鸣桃源风雅颂-2-准业主2', 9, 0],
 ['徐州淮海壹号-2-准业主2', 55, 1],
 ['徐州玉兰公馆-2-准业主2', 62, 3],
 ['枣庄御园-1-准业主1', 1, 1],
 ['枣庄御园-2-准业主2', 53, 5],
 ['常州滨湖御园-1-准业主1', 11, 1],
 ['常州滨湖御园-2-准业主2', 130, 2],
 ['靖江玉兰公馆-1-准业主1', 10, 0],
 ['靖江玉兰公馆-2-准业主2', 126, 7],
 ['苏州泰兴御园-1-准业主1', 15, 2],
 ['苏州泰兴御园-2-准业主2', 19, 0],
 ['泰兴桃源府南苑-2-准业主2', 7, 0],
 ['泰州融创玉兰花园-1-准业主1', 12, 0],
 ['泰州融创玉兰花园-2-准业主2', 11,

In [9]:
"""Get city info
    :return format: {CITY: [total, wrong]}
"""
def analyze(l: list):
    res = {}
    for i in range(len(l)):
        city = l[i][0].split('-')[0]
        if not res.__contains__(city):
            res[city] = [l[i][1], l[i][2]]
        else:
            res[city][0] += l[i][1]
            res[city][1] += l[i][2]
    return res

"""Calculate wrong rate
    :return format: {CITY: [total, wrong, wrong rate]}
"""
def calculate(d: dict):
    for k in d.keys():
        print(k, d[k])
        d[k].append(float('{:.5f}'.format(d[k][1]/d[k][0])))
    return d

In [10]:
zyz_list = []
mhq_list = []
wdq_list = []
lyz_list = []
for k, v in calculate(analyze(zyz)).items():
    zyz_list.append(tuple([k, '准业主'] + v))
for k, v in calculate(analyze(mhq)).items():
    mhq_list.append(tuple([k, '磨合期'] + v))
for k, v in calculate(analyze(wdq)).items():
    wdq_list.append(tuple([k, '稳定期'] + v))
for k, v in calculate(analyze(lyz)).items():
    lyz_list.append(tuple([k, '老业主'] + v))

滁州凯迪融创玉兰公馆 [67, 1]
滁州凯迪融创御园 [13, 0]
南京融创中南御园 [1, 0]
南通江语海 [4, 0]
南通九里桃源 [126, 0]
南通玖樟台 [17, 0]
南通玉兰公馆 [1, 0]
启东海上桃源 [4, 0]
启东桃源里 [8, 1]
呼和浩特瀚海壹號 [18, 0]
上海桃花源 [1, 0]
上海葛洲坝虹桥玫瑰公馆 [19, 0]
上海领馆壹号院 [1, 0]
上海山水拾间 [14, 0]
上海壹号公馆 [5, 0]
淮安融创广场 [4, 0]
淮南玉兰公馆 [16, 0]
徐州枫丹公馆 [54, 3]
徐州凤鸣桃源 [1, 0]
徐州凤鸣桃源风雅颂 [9, 0]
徐州淮海壹号 [55, 1]
徐州玉兰公馆 [62, 3]
枣庄御园 [54, 6]
常州滨湖御园 [141, 3]
靖江玉兰公馆 [136, 7]
苏州泰兴御园 [34, 2]
泰兴桃源府南苑 [7, 0]
泰州融创玉兰花园 [23, 0]
无锡大塘御园 [104, 1]
无锡枫丹御园 [12, 2]
无锡惠山国际社区 [61, 0]
无锡惠山映 [3, 0]
无锡玖里映月 [4, 0]
无锡文旅城 [20, 1]
无锡西羲里 [8, 0]
无锡壹号院 [6, 0]
无锡运河壹号府 [130, 2]
无锡长江映 [1, 1]
镇江观澜公馆 [132, 5]
镇江御园 [23, 0]
昆山玉兰公馆 [70, 4]
苏州泊云庭 [3, 0]
苏州浦西玫瑰园 [31, 2]
苏州壹号院 [2, 0]
乌鲁木齐江南桃源 [40, 0]
上海桃花源 [3, 0]
上海香溢花城 [1, 0]
上海滨江壹号院 [1, 1]
常州御园一二三期 [8, 0]
江阴江南御园 [13, 0]
无锡瑷颐湾 [14, 0]
无锡东方御园 [1, 0]
无锡江阴玉兰公馆 [8, 0]
无锡满堂院 [5, 0]
无锡森邻森邻 [6, 0]
无锡香樟园 [185, 18]
无锡玉兰公馆 [36, 1]
无锡运河壹号府 [270, 7]
芜湖玉兰公馆 [45, 5]
昆山玉兰公馆 [1, 0]
苏州太湖御园 [115, 4]
苏州壹号院 [4, 1]
张家港滨江御园 [5, 0]


In [11]:
labels = ['城市', '业主阶段', '接触量', '错号量', '错号率']
zyz_xlxs = pd.DataFrame(data=zyz_list, columns=labels)
zyz_xlxs = zyz_xlxs.sort_values(by='错号率', ascending=False)
zyz_xlxs.head(20)

,城市,业主阶段,接触量,错号量,错号率
37,无锡长江映,准业主,1,1,1.00000
29,无锡枫丹御园,准业主,12,2,0.16667
8,启东桃源里,准业主,8,1,0.12500
22,枣庄御园,准业主,54,6,0.11111
42,苏州浦西玫瑰园,准业主,31,2,0.06452
25,苏州泰兴御园,准业主,34,2,0.05882
40,昆山玉兰公馆,准业主,70,4,0.05714
17,徐州枫丹公馆,准业主,54,3,0.05556
24,靖江玉兰公馆,准业主,136,7,0.05147
33,无锡文旅城,准业主,20,1,0.05000


In [12]:
mhq_xlxs = pd.DataFrame(data=mhq_list, columns=labels)
mhq_xlxs = mhq_xlxs.sort_values(by='错号率', ascending=False)
mhq_xlxs.head(20)

,城市,业主阶段,接触量,错号量,错号率
2,上海滨江壹号院,磨合期,1,1,1.00000
16,苏州壹号院,磨合期,4,1,0.25000
13,芜湖玉兰公馆,磨合期,45,5,0.11111
10,无锡香樟园,磨合期,185,18,0.09730
15,苏州太湖御园,磨合期,115,4,0.03478
11,无锡玉兰公馆,磨合期,36,1,0.02778
12,无锡运河壹号府,磨合期,270,7,0.02593
0,上海桃花源,磨合期,3,0,0.00000
14,昆山玉兰公馆,磨合期,1,0,0.00000
9,无锡森邻森邻,磨合期,6,0,0.00000


In [13]:
wdq_xlxs = pd.DataFrame(data=wdq_list, columns=labels)
wdq_xlxs = wdq_xlxs.sort_values(by='错号率', ascending=False)
wdq_xlxs.head(20)

,城市,业主阶段,接触量,错号量,错号率


In [14]:
lyz_xlxs = pd.DataFrame(data=lyz_list, columns=labels)
lyz_xlxs = lyz_xlxs.sort_values(by='错号率', ascending=False)
lyz_xlxs.head(20)

,城市,业主阶段,接触量,错号量,错号率


In [15]:
# Shanghai
def getShanghaiInfo(raw, zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs):
    for _, rows in raw.iterrows():
        if '准业主' in rows['行标签']:
            # tuple = ('上海区域', '准业主', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量'])))
            # print(('上海区域', '准业主', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量']))))
            shzyz = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['准业主'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
            zyz_xlxs = pd.concat([zyz_xlxs, shzyz])

        if '磨合期' in rows['行标签']:
            shmhq = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['磨合期'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})

            mhq_xlxs = pd.concat([mhq_xlxs, shmhq])
        if '稳定期' in rows['行标签']:
            shwdq = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['稳定期'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
            wdq_xlxs = pd.concat([wdq_xlxs, shwdq])
        if '老业主' in rows['行标签']:
            shlyz = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['老业主'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
            lyz_xlxs = pd.concat([lyz_xlxs, shlyz])
    return [zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs]

dflist = [zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs]
try:
    df = pd.read_excel(FILENAME, sheet_name='上海区域')
    raw = df.copy(deep=True)
    simplify(raw)
    dflist = getShanghaiInfo(raw, zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs)
except Exception as e:
    print(e)
    # print("No such sheet!")
    pass

No sheet named <'上海区域'>


In [16]:
# Excel weiter funtion
def multiple_dfs(df_list, sheets, file_name, spaces):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    row = 0
    for dataframe in df_list:
        dataframe.to_excel(writer, sheet_name=sheets, startrow=row, startcol=0, index=False)   
        row = row + len(dataframe.index) + spaces + 1
    writer.save()

# write list of dataframes
import time
file_name = FILENAME.strip().split('/')[-1].strip().split('.')[0]+'-'+time.strftime("%Y%m%d", time.localtime())+'.xlsx'
multiple_dfs(dflist, 'Validation', file_name, 1)
print("File processed completed!")

File processed completed!
